In [1]:
#%pip install openpyxl
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas as pd
import numpy as np
import math
import nltk
import string

remove_punct_map = dict.fromkeys(map(ord, string.punctuation))
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)
from itertools import compress
from nltk import word_tokenize
import re

In [2]:
from nltk.corpus import stopwords
esp_stop = stopwords.words('spanish')
esp_stop = esp_stop + ['ser','tener','deber']
esp_stop.remove('estado')
esp_stop.remove('estados')

In [3]:
esp_stop

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [4]:
import spacy
nlp = spacy.load('es_core_news_sm')

def lemmatizer(text):  
  doc = nlp(text)
  #return ' '.join([word.lemma_ for word in doc if word.lemma_ not in esp_stop])
  return ' '.join([word.text for word in doc if word.text not in esp_stop])

C:\Users\aleja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ..\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92


https://github.com/dccuchile/spanish-word-embeddings


### Carga y pre-procesamiento

In [5]:
cabildos = pd.read_excel('data/resultadocabildoprovincial.xlsx')

In [6]:
cabildos.head()

,region_nombre,provincia_nombre,comuna_nombre,cabildo_id,cabildo_nombre,cabildo_direccion,num_mesa,pregunta_glosa,concepto_nombre,concepto_otro,tipo_eleccion,fundamento
0,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Autonomía/libertad,NaN,A,sin fundamento
1,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Sin fundamento
2,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Descentralización,NaN,A,La descentralizacion de ser autonoma por ser z...
3,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Equidad de género,NaN,A,Sin fundamento
4,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Respeto/conservación de la naturaleza o medio ...,NaN,A,Sin fundamento


In [7]:
cabildos.sample(30)

,region_nombre,provincia_nombre,comuna_nombre,cabildo_id,cabildo_nombre,cabildo_direccion,num_mesa,pregunta_glosa,concepto_nombre,concepto_otro,tipo_eleccion,fundamento
1091,Tarapacá,Iquique,Iquique,106,Colegio Inglés,"Patricio Lynch 1580, Iquique",3,¿Cuáles son los deberes y responsabilidades qu...,Respeto por la constitución,NaN,A,Mantener el orden dentro de la sociedad
1961,Antofagasta,Antofagasta,Antofagasta,79,Colegio Don Bosco,"Huamachuco 9567, Antofagasta",9,"¿Cuáles son los derechos, que la Constitución ...",Libertad de expresión,NaN,A,REGULACION DE ACCESO A LOS MEDIOS DE COMUNICAC...
3964,Coquimbo,Elqui,Coquimbo,103,Liceo Diego Portales (A-8),"Francisco Varela 950, Coquimbo",3,¿Cuáles son las instituciones del Estado que d...,Otro,ASAMBLEA CONSTITUYENTE,A,EN CATEGORÍA DE OTRAS INSTITUCIONES ESTE INSTR...
6366,Valparaiso,San Antonio,San Antonio,125,Colegio Pedro Aguirre Cerda (ex Grupo Escalar),"Av. Barros Luco 1945, Barrancas, San Antonio",6,¿Cuáles son los valores y principios más impor...,Respeto/conservación de la naturaleza o medio ...,NaN,A,Conservación flora y fauna nativa. política qu...
44,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",2,"¿Cuáles son los derechos, que la Constitución ...",Otro,derecho a elegir y ser elegido a cargos publicos,A,sin fundamento
154,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",5,"¿Cuáles son los derechos, que la Constitución ...",Respeto a la naturaleza/medio ambiente,NaN,A,Medio ambiente grato\nAire puro/agua pura\nRes...
5231,Valparaiso,Isla de Pascua,Isla de Pascua,108,Colegio Lorenzo Baeza,"Te Pito O Te Henua S/N, Isla de Pascua",2,"¿Cuáles son los derechos, que la Constitución ...",A la educación,NaN,A,Sin fundamento
2816,Atacama,Copiapó,Copiapó,98,Liceo Música,"Andrés Bello 170, Copiapó",7,"¿Cuáles son los derechos, que la Constitución ...",A la participación,NaN,A,en la toma de decisiones locales y nacionales....
5550,Valparaiso,Petorca,La Ligua,122,Liceo Pulmahue Municipal de la Ligua,"Pedro Polanco #480, La Ligua",2,¿Cuáles son las instituciones del Estado que d...,Gobierno regional,NaN,A,Mayor autonomía y poder para descentralizar el...
14914,Metropolitana de Santiago,Santiago,Independencia,130,Liceo Gabriela Mistral,"Independencia 1225, Independencia",9,¿Cuáles son los valores y principios más impor...,Solidaridad,NaN,P,Sin fundamentos.


In [8]:
cabildos.shape

(30414, 12)

Pre-procedimiento (estándar) para un método BoW

In [9]:
cabildos['fundamento'] = cabildos['fundamento'].str.lower()
cabildos['fundamento'] = cabildos['fundamento'].apply(lambda x: re.sub("[0-9]", '', x))
cabildos['fundamento'] = cabildos['fundamento'].str.translate(str.maketrans(' ', ' ', string.punctuation))
cabildos['fundamento'] = cabildos['fundamento'].str.replace('\n',' ')
cabildos['fundamento'] = cabildos['fundamento'].str.strip()


cabildos = cabildos[[len(x.split( )) >3  for x in cabildos.fundamento]]
cabildos = cabildos.reset_index(drop=True)

In [10]:
cabildos.shape

(18530, 12)

### Con gensim

Vamos a partir por entrenar un modelo con el corpus que tenemos. 

In [11]:
import gensim
from gensim import corpora
from gensim import models

Hay que crear primero el corpus, una vez tenemos la data limpia.

In [12]:
corpus = [word_tokenize(lemmatizer(texto)) for texto in cabildos.fundamento]

Ahora vamos a entrenar un modelo Word2vec con nuestro corpus. Esto es, vamos a generar un espacio multidimensional que represente las relaciones semánticas de nuestro vocabulario.  

In [13]:
from gensim.models import Word2Vec

Los parámetros de la función Word2Vec

min_count = int - Ignores all words with total absolute frequency lower than this - (2, 100)

window = int - The maximum distance between the current and predicted word within a sentence. E.g. window words on the left and window words on the left of our target - (2, 10)

size = int - Dimensionality of the feature vectors. - (50, 300)

sample = float - The threshold for configuring which higher-frequency words are randomly downsampled. Highly influencial. - (0, 1e-5)

alpha = float - The initial learning rate - (0.01, 0.05)

min_alpha = float - Learning rate will linearly drop to min_alpha as training progresses. To set it: alpha - (min_alpha * epochs) ~ 0.00

negative = int - If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)

workers = int - Use these many worker threads to train the model (=faster training with multicore machines)

In [14]:
model = Word2Vec(corpus, min_count=3, vector_size= 100, window =5)

No es que sea muy útil, pero podemos ver la representación vectorial de alguna palabra del vocabulario...

In [15]:
model.wv['educación']

array([ 0.44064653,  1.115148  ,  0.41125   , -0.873849  ,  0.8057761 ,
       -1.0706223 ,  0.27465472,  1.2555817 , -1.1379077 , -0.6118455 ,
       -0.15992408, -0.48388493,  0.36918446,  0.31732717,  0.5142755 ,
       -1.3999065 ,  0.14710727, -0.32363597,  0.12402956, -1.742387  ,
        0.84484076, -0.5336393 , -0.48934245, -0.9204823 , -0.24985157,
       -0.26756966, -0.21972962, -0.41384184, -0.44419727,  0.2577741 ,
        0.5239204 ,  0.12428603, -0.49943358, -0.8422251 ,  0.5393388 ,
        0.6017225 , -0.67066294, -0.43515828,  0.8701439 , -1.0064807 ,
        0.17805465, -0.7697206 ,  0.1552434 ,  0.0834308 ,  0.4349482 ,
       -0.479556  , -0.6904973 , -0.29377538,  0.45657635,  0.22262135,
        0.4374119 , -0.17426614, -0.63146955,  0.24501702,  0.644543  ,
       -0.00439053,  0.2803969 , -0.28316265,  0.30937484, -0.0904367 ,
        0.1156629 ,  0.28820646, -0.839867  , -0.15678966, -0.43699458,
        1.0481966 ,  0.55382013, -0.01331865, -0.6699637 ,  0.14

Así como podemos estimar la similaridad (coseno) entre las representaciones vectoriales de dos palabras

In [16]:
 model.wv.similarity('educación', 'educacion')

0.9532065

In [17]:
 model.wv.similarity('educación', 'salud')

0.9833009

In [18]:
 model.wv.similarity('educación', 'gratuidad')

0.9633021

In [19]:
 model.wv.similarity('educación', 'trabajo')

0.9045006

Y también podemos buscar las palabras más parecidas a una palabra cualquiera (dentro del vocabulario, claro)

In [20]:
 model.wv.most_similar('educación')

[('calidad', 0.9939150810241699),
 ('laica', 0.9873181581497192),
 ('pública', 0.9853305220603943),
 ('gratuita', 0.9847294092178345),
 ('salud', 0.9833009243011475),
 ('publica', 0.97789466381073),
 ('garantizada', 0.9774911403656006),
 ('universal', 0.9761843085289001),
 ('vivienda', 0.966884970664978),
 ('acceso', 0.9659515619277954)]

Si no nos convence nuestro corpus, otra opción es usar un modelo pre-entrado con un gran corpus. Como el que esta acá

https://github.com/dccuchile/spanish-word-embeddings

Esta parte del código no les correrá porque el modelo pre-entrenado pesa 1GB por lo que no lo subí a la carpeta, pero ustedes pueden bajarlo de ese link si lo quieren usar.

In [21]:
model2 = gensim.models.KeyedVectors.load_word2vec_format('word_embeddings_models/sbw_vectors.bin', binary=True)


In [22]:
 model2.most_similar('educación')

[('enseñanza', 0.7887755632400513),
 ('educativa', 0.7624322175979614),
 ('educativo', 0.7323473691940308),
 ('eduación', 0.7299347519874573),
 ('educativos', 0.7249588966369629),
 ('parvularia', 0.7070096135139465),
 ('preescolar', 0.7016569375991821),
 ('educativas', 0.6875496506690979),
 ('enseñaza', 0.6860960721969604),
 ('posprimaria', 0.6720082759857178)]

In [23]:
 model2.similarity('educación', 'salud')

0.59547806

In [24]:
 model2.similarity('educación', 'educacion')

0.54617745

In [25]:
 model2.similarity('casa', 'educativo')

0.27570283

In [26]:
 model2.most_similar('género')

[('géneros', 0.746495246887207),
 ('subgénero', 0.738937258720398),
 ('genero', 0.6793079376220703),
 ('monotípico', 0.6632034778594971),
 ('Proteuphaedra', 0.6610530018806458),
 ('Gausapia', 0.6553515195846558),
 ('Amerhippus', 0.6463193893432617),
 ('ditrysias', 0.6444892883300781),
 ('Xypetana', 0.6430881023406982),
 ('Tetragnathidae', 0.6430761814117432)]

In [27]:
 model2.most_similar('sexo')

[('heterosexuales', 0.6806257963180542),
 ('heterosexual', 0.6804140210151672),
 ('homosexual', 0.641345202922821),
 ('intercrural', 0.6283185482025146),
 ('transexualidad', 0.6130891442298889),
 ('sexuales', 0.6102189421653748),
 ('homosexuales', 0.6095353364944458),
 ('tribadismo', 0.6065760254859924),
 ('sexual', 0.604019045829773),
 ('heteroflexibilidad', 0.5982562899589539)]

In [28]:
 model.wv.most_similar('sexo')

[('diferencia', 0.9975929856300354),
 ('etnia', 0.9972696304321289),
 ('inclusión', 0.9969273209571838),
 ('géneros', 0.9968518614768982),
 ('generos', 0.9958637952804565),
 ('alguna', 0.9958001971244812),
 ('capacidades', 0.9956372380256653),
 ('raza', 0.9954531788825989),
 ('posibilidades', 0.9947544932365417),
 ('pareja', 0.994751513004303)]

In [29]:
 model2.most_similar('grande')

[('pequeña', 0.6300310492515564),
 ('pequeño', 0.6242244839668274),
 ('mayor', 0.5784777402877808),
 ('importante', 0.5656843185424805),
 ('enorme', 0.564133882522583),
 ('aglorameración', 0.5611023902893066),
 ('giganticus', 0.5526046752929688),
 ('gran', 0.5491284728050232),
 ('Silmido', 0.5491080284118652),
 ('gigantesco', 0.5438647270202637)]

In [30]:
 model2.most_similar('gato')

[('perro', 0.7445881366729736),
 ('zorro', 0.7061581611633301),
 ('conejo', 0.7018613815307617),
 ('montés', 0.6875571012496948),
 ('mapache', 0.6867462396621704),
 ('maúlla', 0.6719425916671753),
 ('tigre', 0.6647046804428101),
 ('lybica', 0.6631762385368347),
 ('huiña', 0.6606248617172241),
 ('gatito', 0.6600393056869507)]

In [31]:
vect1 = model2['rey'] - model2['hombre'] + model2['mujer']

In [32]:
vect2 = model2['reina']

In [33]:
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
cosine_similarity([vect1],[vect2])

array([[0.7737196]], dtype=float32)

In [35]:
 model2.similarity('rey', 'hombre')

0.309018

In [36]:
 model2.similarity('hombre', 'mujer')

0.71744835

In [37]:
 model2.similarity('rey', 'reina')

0.72538036

In [38]:
 model2.most_similar('reina')

[('rey', 0.7253803610801697),
 ('princesa', 0.7232583165168762),
 ('consorte', 0.7053413391113281),
 ('Ranavalona', 0.6801236271858215),
 ('monarca', 0.6759358048439026),
 ('Rasoherina', 0.6722511649131775),
 ('reinas', 0.6713809370994568),
 ('Reina', 0.6703052520751953),
 ('Isabel', 0.6599754691123962),
 ('Kottanner', 0.6597455143928528)]

### Clasificación de textos con Word2vec


In [39]:
reclamos = pd.read_excel('data/reclamos.xlsx')

In [40]:
reclamos.head()

,CASO_ID,MERCADO_ANALISTA,DESCRIPCION_CIUDADANO,PETICION_CIUDADANO
0,638428,APIA -Reclamo Seguros,RECIBI UN CORREO ELECTRONICO DONDE SE ME INDIC...,DADO QUE NINGUNA DE LAS CAUSALES ANTERIORES SE...
1,580194,APIA -Reclamo Seguros,Mi nombre es XX y el día 27 de Enero de este a...,Necesito que Metlife me deposite el dinero sol...
2,447321,APIA -Reclamo Seguros,Estimados Señores SVS\n\nCon fecha 13 de Octub...,Por esta razón quiero colocar un reclamo forma...
3,437201,APIA -Reclamo Seguros,En primera instancia solicité renvar mi tarjet...,Solicito devolución de lo que me cobraron por ...
4,385926,Reclamo Valores,reclamo en contra Scotiabank por cobros de man...,condonacion de los cobros que hace el Banco.


In [41]:
reclamos['DESCRIPCION_CIUDADANO'] = reclamos['DESCRIPCION_CIUDADANO'].str.lower()
reclamos['DESCRIPCION_CIUDADANO'] = reclamos['DESCRIPCION_CIUDADANO'].apply(lambda x: re.sub("[0-9]", '', x))
reclamos['DESCRIPCION_CIUDADANO'] = reclamos['DESCRIPCION_CIUDADANO'].str.translate(str.maketrans(' ', ' ', string.punctuation))
reclamos['DESCRIPCION_CIUDADANO'] = reclamos['DESCRIPCION_CIUDADANO'].str.replace('\n',' ')
reclamos['DESCRIPCION_CIUDADANO'] = reclamos['DESCRIPCION_CIUDADANO'].str.strip()


reclamos = reclamos[[len(x.split( )) >2  for x in reclamos.DESCRIPCION_CIUDADANO]]
reclamos = reclamos.reset_index(drop=True)

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
text_train, text_test, y_train, y_test = train_test_split(reclamos['DESCRIPCION_CIUDADANO'], reclamos['MERCADO_ANALISTA'], random_state=42, test_size=0.25)

In [44]:
corpus_train = [word_tokenize(lemmatizer(texto)) for texto in text_train]

In [45]:
w2v_model = gensim.models.Word2Vec(corpus_train, vector_size=100, window=5, min_count=2)

In [46]:
words = w2v_model.wv.index_to_key

In [47]:
len(words)

4620

In [48]:
words

['seguro',
 'respuesta',
 'fecha',
 'xx',
 'seguros',
 'día',
 'siniestro',
 'compañía',
 'si',
 'banco',
 'póliza',
 'pago',
 'año',
 'aseguradora',
 'reclamo',
 'mes',
 'empresa',
 'parte',
 'vehículo',
 'solo',
 'cuenta',
 'carta',
 'meses',
 'hace',
 'días',
 'hacer',
 'mas',
 'auto',
 'contrato',
 'hoy',
 'correo',
 'información',
 'caso',
 'luego',
 'acciones',
 'liquidador',
 'años',
 'taller',
 'dos',
 'uf',
 'presente',
 'monto',
 'mismo',
 'valor',
 'dinero',
 'crédito',
 'momento',
 'adjunto',
 'embargo',
 'poliza',
 'pagar',
 'indica',
 'informe',
 'después',
 'daños',
 'ahora',
 'solicitud',
 'fondos',
 'nunca',
 'aun',
 'octubre',
 'cobertura',
 'problema',
 'forma',
 'julio',
 'situación',
 'según',
 'cuotas',
 'dia',
 'tiempo',
 'persona',
 'hecho',
 'estado',
 'hice',
 'diciembre',
 'nuevamente',
 'marzo',
 'abril',
 'ejecutiva',
 'poder',
 'ninguna',
 'plazo',
 'vida',
 'total',
 'dice',
 'corredora',
 'pasado',
 'además',
 'condiciones',
 'través',
 'svs',
 'asegurad

Ahora generamos la representación vectorial para cada palabra de cada frase de los sets de entrenamiento y prueba

In [49]:
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])  for ls in text_train], dtype="object")


In [50]:
X_test_vect  = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])  for ls in text_test], dtype="object")

In [51]:
# Tenemos un arreglo de arreglos, 749 documentos, cada uno con un número distinto de palabras
X_train_vect.shape

(749,)

In [52]:
# Veamos el documento en el índice 10, este tiene 165 palabras, cada una representada por un vector de largo 100
X_train_vect[10].shape

(165, 100)

Tenemos entonces un array, donde para cada elemento documento, hay una representación vectorial para cad apalabra del documento. Lo que haremos ahora es promediar las representaciones vectoriales para cada documento.

In [53]:
# de https://medium.com/@dilip.voleti/classification-using-word2vec-b1d79d375381
X_train_vect_avg = []
for v in X_train_vect:
   X_train_vect_avg.append(v.mean(axis=0))
        
X_test_vect_avg = []
for v in X_test_vect:
   X_test_vect_avg.append(v.mean(axis=0))


Vamos a entrenar un modelo de clasificación Random Forest para ver a que tipo de reclamo (valores o seguros) corresponde cada documento:

In [54]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(X_train_vect_avg, y_train.values.ravel())

Ahora vamos a aplicar el modelo entrenado en los documentos de test:

In [55]:
y_test_result = rf_model.predict(X_test_vect_avg)

In [56]:
print(y_test_result[0:10])
print(y_test[0:10])

['APIA -Reclamo Seguros ' 'APIA -Reclamo Seguros '
 'APIA -Reclamo Seguros ' 'APIA -Reclamo Seguros ' 'Reclamo Valores'
 'APIA -Reclamo Seguros ' 'APIA -Reclamo Seguros '
 'APIA -Reclamo Seguros ' 'APIA -Reclamo Seguros '
 'APIA -Reclamo Seguros ']
453           Reclamo Valores
793    APIA -Reclamo Seguros 
209    APIA -Reclamo Seguros 
309           Reclamo Valores
740           Reclamo Valores
578           Reclamo Valores
895    APIA -Reclamo Seguros 
545    APIA -Reclamo Seguros 
436    APIA -Reclamo Seguros 
678           Reclamo Valores
Name: MERCADO_ANALISTA, dtype: object


Veamos la performance general del modelo para el grupo de documentos que separamos para el test

In [57]:
rf_model.score(X_test_vect_avg, y_test)

0.72